<a href="https://colab.research.google.com/github/MarcusStafford-MLS/marcus-stafford.github.io/blob/main/world_bank_info_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code for challenge exercise goes here
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

variable_options = [
 ('Total Population', 'SP.POP.TOTL'),
 ('Population Living in Slums', 'EN.POP.SLUM.UR.ZS'),
 ('Literacy Rate, adult population', 'SE.ADT.LITR.ZS'),
 ('Population Density by people per sq. km', 'EN.POP.DNST'),
 ('Imports of goods and services', 'NE.IMP.GNFS.ZS'),
 ('Tax Revenue, as a percentage of GDP', 'GC.TAX.TOTL.GD.ZS'),
 ('Percentage of Total Exports', 'NE.EXP.GNFS.ZS'),
 ('Electricity access, as a pecentage of the total population', 'EG.ELC.ACCS.ZS'),
 ('Literacy rate, youth population', 'SE.ADT.1524.LT.ZS'),
 ('Birth rate, per 1,000', 'SP.DYN.CBRT.IN')]

variable_dropdown = widgets.Dropdown(
    options = variable_options,
    description = 'Variable:',
    disabled = False)

color_options = ['Purples', 'Blues', 'Greens', 'Greys', 'Oranges']
colors_dropdown = widgets.Dropdown(
    options = color_options,
    description = 'Color:',
    disabled = False,)

year_options = [2020,2019,2018,2017,2016]
years_dropdown = widgets.Dropdown(
    options = year_options,
    description='Year:',
    disabled=False,)

grid = widgets.GridspecLayout(1,3,height='60px')
grid[0,0] = years_dropdown
grid[0,1] = variable_dropdown
grid[0,2] = colors_dropdown

map_plot = None
def map_update(event=None):
  global map_plot
  clear_output(wait=True)
  display(grid)

  url_get = 'https://api.worldbank.org/v2/country/all/indicator/' + str(variable_dropdown.value) + '?format=json&per_page=500&date=' + str(years_dropdown.value)
  response_ = requests.get(url_get)
  r_json = response_.json()
  df = pd.DataFrame(r_json[1])
  df_sub = df[['countryiso3code','value']]

  world_map = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
  merged_df_ = world_map.merge(df_sub, left_on = 'iso_a3', right_on = 'countryiso3code')

  ax = merged_df_.plot(column = 'value',scheme='quantiles', legend = True, legend_kwds={'loc': 'lower left'}, cmap=colors_dropdown.value, figsize=(14,10))
  merged_df_.hist('value', color='Red', alpha=0.8, edgecolor='black', figsize = (10,4))
  plt.title('Histogram of Values')

map_update()
years_dropdown.observe(map_update,names = 'value')
variable_dropdown.observe(map_update,names = 'value')
colors_dropdown.observe(map_update, names = 'value')